## Import Libraries

In [69]:
import time
import rospy
import numpy as np
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
from dual_quaternion import Quaternion
from dual_quaternion import DualQuaternion
from casadi import Function

## Working with Symbolic Variables

In [70]:
# Defining Mobile Robo Frame
theta_1 = ca.SX.sym('theta_1', 1, 1)
n_1 = ca.SX([0.0, 0.0, 1.0])
q_1 = ca.vertcat(ca.cos(theta_1/2), ca.sin(theta_1/2)@n_1)
tx_1 = ca.SX.sym("x_1", 1, 1)
ty_1 = ca.SX.sym("y_1", 1, 1)
t_1 = ca.vertcat(0.0, tx_1, ty_1, 0.0)

# Time Derivaties
theta_1p = ca.SX.sym('theta_1p', 1, 1)
tx_1p = ca.SX.sym("x_1p", 1, 1)
ty_1p = ca.SX.sym("y_1p", 1, 1)

aux_symbolic_total = ca.vertcat(tx_1, ty_1, theta_1)
symbolic_actions = ca.vertcat(tx_1, ty_1, theta_1)

control = ca.vertcat(tx_1p, ty_1p, theta_1p)

In [71]:

# Defining Desired Frame
theta_1d = ca.SX.sym('theta_1d', 1, 1)
n_1d = ca.SX([0.0, 0.0, 1.0])
q_1d = ca.vertcat(ca.cos(theta_1d/2), ca.sin(theta_1d/2)@n_1d)
tx_1d = ca.SX.sym("x_1d", 1, 1)
ty_1d = ca.SX.sym("y_1d", 1, 1)
t_1d = ca.vertcat(0.0, tx_1d, ty_1d, 0.0)

# Time Derivaties
theta_1pd = ca.SX.sym('theta_1pd', 1, 1)
tx_1pd = ca.SX.sym("x_1pd", 1, 1)
ty_1pd = ca.SX.sym("y_1pd", 1, 1)

### Computing Sequential Transformations

In [72]:
Q1 = DualQuaternion.from_pose(quat = q_1, trans = t_1)
Q1.get_dual

Quaternion: @1=0, @2=0.5, @3=2, @4=cos((theta_1/@3)), @5=sin((theta_1/@3)), [@1, (@2*((x_1*@4)+(y_1*@5))), (@2*((y_1*@4)-(x_1*@5))), @1]

In [73]:
Q1d = DualQuaternion.from_pose(quat = q_1d, trans = t_1d)


## Generating Function Using CasADi

In [74]:
Function_DualQuat_total = Function('DualQuat_total', [aux_symbolic_total], [Q1.get[:, 0]])
Function_DualQuat_total_trans = Function('DualQuat_total_trans', [aux_symbolic_total], [Q1.get_trans.get[:, 0]])
Function_DualQuat_total_quat = Function('DualQuat_total_quat', [aux_symbolic_total], [Q1.get_quat.get[:, 0]])

## Using Functions

In [75]:
input_t = np.array([-2, 5, np.pi/2])
dual = Function_DualQuat_total(input_t)
dual

DM([0.707107, 0, 0, 0.707107, 0, 1.06066, 2.47487, 0])

In [76]:
input_t = np.array([-2, 5, np.pi/2])
trans = Function_DualQuat_total_trans(input_t)
trans

DM([0, -2, 5, 0])

In [77]:
input_t = np.array([-2, 5, np.pi/2])
quat = Function_DualQuat_total_quat(input_t)
quat

DM([0.707107, 0, 0, 0.707107])

### Calculating Jacobians

In [78]:
J = ca.jacobian(Q1.get[:, 0], symbolic_actions)
J.shape

(8, 3)

In [79]:
Function_J = Function("Function_J", [aux_symbolic_total], [J])

In [80]:
input_t = np.array([5, 5, 0.55])
Jacobian = Function_J(input_t)
print(Jacobian)


[[00, 00, -0.135773], 
 [00, 00, 00], 
 [00, 00, 00], 
 [00, 00, 0.481213], 
 [00, 00, 00], 
 [0.481213, 0.135773, 0.863598], 
 [-0.135773, 0.481213, -1.54247], 
 [00, 00, 00]]


## Computing Differential Kinematics

In [81]:
diff_kinematics = J @ control

Function_diff_kinematics = Function("Function_diff_kinematics", [aux_symbolic_total, control], [diff_kinematics])

In [82]:
input_t = np.array([-3.15, 0.0, 1])
input_t_p = np.array([0.36, 0.0, 0.0])
Xp = Function_diff_kinematics(input_t, input_t_p)
Xp

DM([-0, 00, 00, 0, 00, 0.157965, -0.0862966, 00])